## Прогнозирование с помощью регрессии

In [1]:
%pylab inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

import scipy.stats as ss
from scipy import signal
import scipy as sc

from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_error as MAE
import statsmodels.formula.api as smf

import pickle

Populating the interactive namespace from numpy and matplotlib


Загружаем данные о количестве поездок в отобранных регионах

In [2]:
with open("./data/regions.pkl", "rb") as inf:
    data = pickle.load(inf)

In [3]:
data.transpose().shape

(21888L, 102L)

In [4]:
with open("./data/names.pkl", "rb") as inf:
    regions = np.array(pickle.load(inf)) + 1

In [5]:
dates = pd.date_range('2014-01-01 00:00:00', periods=data.shape[1], freq='H')
df = pd.DataFrame(data.transpose(), index=dates, columns=regions)
df.head()

,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
2014-01-01 00:00:00,87.0,146.0,70.0,113.0,367.0,645.0,589.0,799.0,948.0,321.0,...,9.0,0.0,5.0,89.0,10.0,35.0,9.0,106.0,22.0,71.0
2014-01-01 01:00:00,92.0,184.0,93.0,153.0,539.0,604.0,490.0,635.0,667.0,225.0,...,24.0,0.0,3.0,22.0,2.0,5.0,0.0,87.0,0.0,44.0
2014-01-01 02:00:00,108.0,165.0,55.0,151.0,443.0,571.0,465.0,499.0,455.0,124.0,...,27.0,0.0,3.0,23.0,1.0,1.0,0.0,39.0,0.0,1.0
2014-01-01 03:00:00,77.0,108.0,32.0,112.0,372.0,533.0,442.0,370.0,307.0,101.0,...,57.0,0.0,0.0,3.0,2.0,1.0,0.0,5.0,1.0,0.0
2014-01-01 04:00:00,47.0,79.0,22.0,77.0,213.0,383.0,296.0,319.0,261.0,87.0,...,38.0,0.0,1.0,9.0,1.0,8.0,0.0,29.0,1.0,18.0


In [6]:
df_s = df.loc['2016-03-01 00:00:00':'2016-05-31 17:00:00']
df_s_f = df.loc['2016-03-01 00:00:00':'2016-05-31 23:00:00']
df_s.head()

,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
2016-03-01 00:00:00,14.0,22.0,20.0,39.0,78.0,124.0,183.0,193.0,212.0,31.0,...,4.0,0.0,1.0,105.0,16.0,59.0,20.0,146.0,12.0,120.0
2016-03-01 01:00:00,16.0,15.0,12.0,12.0,58.0,60.0,98.0,122.0,144.0,14.0,...,5.0,0.0,2.0,5.0,1.0,33.0,9.0,71.0,6.0,23.0
2016-03-01 02:00:00,4.0,14.0,0.0,9.0,23.0,55.0,59.0,93.0,162.0,21.0,...,10.0,1.0,0.0,0.0,0.0,1.0,0.0,18.0,0.0,0.0
2016-03-01 03:00:00,2.0,7.0,1.0,4.0,16.0,35.0,39.0,63.0,119.0,11.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,1.0
2016-03-01 04:00:00,5.0,16.0,2.0,8.0,30.0,31.0,25.0,44.0,79.0,4.0,...,7.0,0.0,0.0,4.0,1.0,0.0,0.0,34.0,0.0,1.0


Функции для моделирования сезонностей и создания dummy переменных

In [7]:
def make_fourier_regressors(data_f, num_end):
    str_var = ''
    length = data_f.shape[0]
    for i in range(1, num_end+1):
        sin = "s_" + str(i)
        cos = "c_" + str(i)
        data_f[sin] = np.sin(2*np.pi*i*np.arange(1, length+1)/168.0)
        data_f[cos] = np.cos(2*np.pi*i*np.arange(1, length+1)/168.0)
        str_var = str_var + sin + ' + '
        if i != num_end:
            str_var = str_var + cos + ' + '
        else:
            str_var = str_var + cos
    return str_var

def make_dummy_weekday(data):
    data['monday'] = [1 if date.weekday() == 0 else 0 for date in data.index]
    data['tuesday'] = [1 if date.weekday() == 1 else 0 for date in data.index]
    data['wednessday'] = [1 if date.weekday() == 2 else 0 for date in data.index]
    data['thursday'] = [1 if date.weekday() == 3 else 0 for date in data.index]
    data['friday'] = [1 if date.weekday() == 4 else 0 for date in data.index]
    data['saturday'] = [1 if date.weekday() == 5 else 0 for date in data.index]
    data['sunday'] = [1 if date.weekday() == 6 else 0 for date in data.index]
    weekday_str = ' + tuesday + wednessday + thursday + friday + saturday + sunday'
    return weekday_str

def fourier_prediction(data, train_time_limit, degree=49):
    data_c = pd.DataFrame(data.values, columns = ['val'], index = data.index)
    str_reg = 'val ~ '
    week_day_str, str_var = '', ''
    str_var = make_fourier_regressors(data_c, degree)
    week_day_str = make_dummy_weekday(data_c)
    model = smf.ols(str_reg + str_var + week_day_str, data=data_c.loc[:train_time_limit])
    fitted = model.fit(cov_type='HC1')
    return fitted.predict(data_c)

In [8]:
def data_construction(data, data_f, train_time_limit):
   
    K = 12 # number of hour lags
    K_d = 7 # number of daily lags
    K_f = 49 # number of fourier components

    # data construction
    offset = K_d*24
    train_num_limit = []
    test_num_start = data.loc[:train_time_limit].shape[0] - 1
    df_res = []
    for data_num in range(1, 7):
        train_num_limit.append(data.loc[:train_time_limit].shape[0] - data_num)
    length = data.shape[0]
    for column in data.columns:
        train_df_list = []
        for data_num in range(6):
            train_df_list.append(pd.DataFrame())
        test_df = pd.DataFrame()
        test_fourier = pd.DataFrame()
       
        for col in data.columns:
            for data_num in range(6):
                if col == column:
                    train_df_list[data_num]['region'+str(col)] = [1]*(train_num_limit[data_num] - offset)
                else:
                    train_df_list[data_num]['region'+str(col)] = [0]*(train_num_limit[data_num] - offset)
            if col == column:
                test_df['region'+str(col)] = [1]*(length - test_num_start)
            else:
                test_df['region'+str(col)] = [0]*(length - test_num_start)
        
        for data_num in range(6):
            train_df_list[data_num]['region'] = [column]*(train_num_limit[data_num] - offset)
        test_df['region'] = [column]*(length - test_num_start)
       
        for h in range(24):
            for data_num in range(6):
                train_df_list[data_num]['hour_'+str(h)] = \
                        map(lambda x: 1 if x.hour == h else 0, data.iloc[offset:train_num_limit[data_num]].index)
            test_df['hour_'+str(h)] = map(lambda x: 1 if x.hour == h else 0, data.iloc[test_num_start:].index)
        
        # Day
        for h in range(7):
            for data_num in range(6):
                train_df_list[data_num]['day_'+str(h)] = \
                        map(lambda x: 1 if x.weekday() == h else 0, data.iloc[offset:train_num_limit[data_num]].index)
            test_df['day_'+str(h)] = map(lambda x: 1 if x.weekday() == h else 0, data.iloc[test_num_start:].index)
        # Value
        for data_num in range(6):
            train_df_list[data_num]['val'] = data.iloc[offset:train_num_limit[data_num]][column].values
        test_df['val'] = data.iloc[test_num_start:][column].values
        for ind in range(1, K+1):
            for data_num in range(6):
                train_df_list[data_num]['val_'+str(ind)] = \
                        data.iloc[offset-ind:train_num_limit[data_num]-ind][column].values
            test_df['val_'+str(ind)] = data.iloc[test_num_start-ind:-ind][column].values
        for ind in range(1, K_d+1):
            for data_num in range(6):
                train_df_list[data_num]['val_d_'+str(ind)] = \
                        data.iloc[offset-24*ind:train_num_limit[data_num]-24*ind][column].values
            test_df['val_d_'+str(ind)] = data.iloc[test_num_start-24*ind:-24*ind][column].values
            
        # Fourier components
        fourier_pred = fourier_prediction(df_s_f[column], '2016-04-30 23:00:00', 49)
        for data_num in range(6):
            (train_df_list[data_num])['fourier'] = fourier_pred[offset + data_num + 1:test_num_start+1].values
            test_fourier['f'+str(data_num)] = fourier_pred[test_num_start + data_num:-6+data_num].values
        test_df['fourier'] = [0]*(test_df.shape[0])
        
        # Target values
        for data_num in range(6):
            train_df_list[data_num]['target'] = data.iloc[offset + data_num + 1:test_num_start+1][column].values
        # Info for submission
        test_df['sub_info'] = df_s.iloc[test_num_start:].index
        # Stacking data
        if column == 1075:
            df_res = train_df_list
            df_test = test_df
            df_test_f = test_fourier
        else:
            for data_num in range(6):
                df_res[data_num] = pd.concat((df_res[data_num], train_df_list[data_num]), axis=0)
            df_test = pd.concat((df_test, test_df), axis=0)
            df_test_f = pd.concat((df_test_f, test_fourier), axis=0)
    for data_num in range(6):
        df_res[data_num].index = range(df_res[data_num].shape[0])
    df_test.index = range(df_test.shape[0])
    df_test_f.index = range(df_test_f.shape[0])
    return df_res, df_test, df_test_f

Будем обучаться на данных с марта 2016 года.

In [9]:
df_s = df.loc['2016-03-01 00:00:00':'2016-05-31 17:00:00']
df_s_f = df.loc['2016-03-01 00:00:00':'2016-05-31 23:00:00']

Построение данных.

In [ ]:
%%time
res, test, test_f = data_construction(df_s, df_s_f, '2016-04-30 23:00:00')

3.Обучение моделей. В качестве регрессионной модели будем использовать ElasticNet. Подберем параметры моделей: alpha и l1_ratio. Первый обозначает множителей регуляризационных членов, а второй - пропорцию между l1 и l2 регуляризаторами.

Подберем оптимальные параметры для каждой регрессионной модели.

In [26]:
def find_bst_params(data_num):
    R, H = 102, 739
    denom = 1.0/(R*H*6)
    alpha_list = np.arange(0.1,1.1,0.2)
    l1_ratio = np.arange(0.1,1.1,0.2)
    bst_score = np.inf
    for alpha in alpha_list:
        for ratio in l1_ratio:
            regressor = ElasticNet(alpha=alpha, l1_ratio=ratio)
            regressor.fit(res[data_num].iloc[:,:-1], res[data_num].target)
            test['fourier'] = test_f['f'+str(data_num)]
            prediction = regressor.predict(test.drop(['sub_info'], axis=1))
            difference = denom*np.abs(prediction - \
                                        df.loc['2016-05-01 0'+str(0+data_num)+':00:00':'2016-05-31 '\
                                               +str(18+data_num)+':00:00'].values.ravel(order='F'))
            err = difference.sum()
            if err < bst_score:
                bst_params = (alpha, ratio)
                bst_score = err
    return bst_params

In [27]:
params_list = []
for data_num in range(6):
    params_list.append(find_bst_params(data_num))

In [28]:
with open("./data/params.pkl", 'wb') as inf:
    pickle.dump(params_list, inf)

Набор оптимальных параметров.

In [29]:
params_list

[(0.10000000000000001, 0.50000000000000011),
 (0.10000000000000001, 0.10000000000000001),
 (0.10000000000000001, 0.10000000000000001),
 (0.90000000000000013, 0.90000000000000013),
 (0.30000000000000004, 0.50000000000000011),
 (0.30000000000000004, 0.50000000000000011)]

Обучаем модели с оптимальными параметрами на данных до апреля.

In [30]:
%%time
# Models training
regressors_list = []
for data_num in range(6):
    regressor = ElasticNet(alpha=params_list[data_num][0], l1_ratio=params_list[data_num][1])
    regressor.fit(res[data_num].iloc[:,:-1], res[data_num].target)
    regressors_list.append(regressor)

Wall time: 1min 52s


Делаем предсказания на май 2016 года и считаем ошибку предсказаний.

In [31]:
%%time
# May predictions
R = 102
H = 739
Q_may = 0
denom = 1.0/(R*H*6)
for data_num in range(6):
    test['fourier'] = test_f['f'+str(data_num)]
    prediction = regressors_list[data_num].predict(test.drop(['sub_info'], axis=1))
    difference = denom*np.abs(prediction - \
                        df.loc['2016-05-01 0'+str(0+data_num)+':00:00':'2016-05-31 '\
                               +str(18+data_num)+':00:00'].values.ravel(order='F'))
    Q_may += difference.sum()

Wall time: 878 ms


In [32]:
Q_may

34.629759364887121

Загружаем соответствующие данные и создаекм выборки для обучения и тестирования.

In [79]:
df_s = df.loc['2016-03-01 00:00:00':'2016-06-30 17:00:00']
df_s_f = df.loc['2016-03-01 00:00:00':'2016-06-30 23:00:00']

In [80]:
%%time
res, test, test_f = data_construction(df_s, df_s_f, '2016-05-31 23:00:00')

CPU times: user 2min 44s, sys: 42.2 s, total: 3min 27s
Wall time: 3min 11s


Обучаем регрессионные модели модели.

In [29]:
%%time
# Models training
regressors_list = []
for data_num in range(6):
    regressor = ElasticNet(alpha=params_list[data_num][0], l1_ratio=params_list[data_num][1])
    regressor.fit(res[data_num].iloc[:,:-1], res[data_num].target)
    regressors_list.append(regressor)

CPU times: user 4min, sys: 38.2 s, total: 4min 39s
Wall time: 2min 54s


In [30]:
all_predictions = []
all_ids = []

Формат данных для отправки результатов.

In [31]:
(test.region.apply(str) + '_' + test.sub_info.apply(lambda x: x.strftime('%Y-%m-%d')) + '_' + \
                   test.sub_info.apply(lambda x: str(x.hour)) + '_' + str(0+1)).head()

0    1075_2016-05-31_23_1
1     1075_2016-06-01_0_1
2     1075_2016-06-01_1_1
3     1075_2016-06-01_2_1
4     1075_2016-06-01_3_1
dtype: object

Делаем предсказания для июня.

In [78]:
%%time
R = 102
H = 715
Q_june = 0
denom = 1.0/(R*H*6)
for data_num in range(6):
    test['fourier'] = test_f['f'+str(data_num)]
    prediction = regressors_list[data_num].predict(test.drop(['sub_info'], axis=1))
    difference = denom*np.abs(prediction - \
                        df.loc['2016-06-01 0'+str(0+data_num)+':00:00':'2016-06-30 '\
                               +str(18+data_num)+':00:00'].values.ravel(order='F'))
    all_predictions.append(prediction)
    all_ids.append(test.region.apply(str) + '_' + test.sub_info.apply(lambda x: x.strftime('%Y-%m-%d')) + '_' + \
                   test.sub_info.apply(lambda x: str(x.hour)) + '_' + str(data_num+1))
    Q_june += difference.sum()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 13.8 µs


In [33]:
print Q_june

33.3109876884


In [113]:
for data_num in range(6):
    if data_num == 0:
        pred_df = pd.DataFrame(all_predictions[data_num], index=all_ids[data_num], columns=['y'])
    else:
        pred_df = pd.concat((pred_df, pd.DataFrame(all_predictions[data_num], index=all_ids[data_num], columns=['y'])),\
                            axis = 0)
pred_df.index.name = 'id'
pred_df.to_csv("submission.csv", sep=',')

In [6]:
R = 102
H = 739
Q_may = 0
denom = 1.0/(R*H*6)

In [33]:
def count_error_region(data, train_time_limit, test_time_limit, region, denom, pred_start='2016-05-01', pred_end='2016-05-31', degree=49,
                       K_d=2, K_h=8):
    new_data = pd.DataFrame(data.loc[:test_time_limit][region].values, columns=['val'], \
                            index = data.loc[:test_time_limit].index)
    error = 0
    all_ids = []
    all_preds = []
    offset = max(24*K_d, K_h, 12)
    # 12-hours 
    new_data['half_day_sum'] = new_data['val'].rolling(12).sum().fillna(0)
    # fourier components
    str_var = make_fourier_regressors(new_data, degree)
    # weekday dummy components
    week_day_str = make_dummy_weekday(new_data)
    # day lags
    for day_lag in range(1, K_d):
        new_data['day_lag_'+str(day_lag)] = [0]*offset + new_data[offset-24*day_lag:-24*day_lag]['val'].values.tolist()
    # hour lags
    for hour_lag in range(1, K_h):
        new_data['hour_lag_'+str(hour_lag)] = [0]*offset + new_data[offset-hour_lag:-hour_lag]['val'].values.tolist()
    # Training and predictions
    for data_num in range(6):
        train_num_limit = data.loc[:train_time_limit].shape[0] - data_num
        regressor = ElasticNet(alpha=0.1, l1_ratio=0.6)
        regressor.fit(new_data.iloc[offset:train_num_limit].drop(['val'], axis=1), \
                      new_data.iloc[offset+data_num+1:train_num_limit+data_num+1].val)
        prediction = regressor.predict(new_data[train_time_limit:].drop(['val'], axis=1))
        difference = denom*np.abs(prediction - \
                        data.loc[pred_start+' 0'+str(0+data_num)+':00:00':pred_end+' '\
                               +str(18+data_num)+':00:00'][region].values)
        error += difference.sum()
        
        indexes = new_data[train_time_limit:].index    
        all_ids.append((pd.Series([str(region)]*indexes.size) + '_' + map(lambda x: x.strftime('%Y-%m-%d'), indexes) + '_' \
                           + map(lambda x: str(x.hour) + '_' + str(data_num+1), indexes)).values)
        all_preds.append(prediction)
    del new_data
    return error, all_preds, all_ids

Предсказание для одного из регионов.

In [35]:
%%time
err, _, _ = count_error_region(df.loc[:'2016-05-31 23:00:00'], '2016-04-30 23:00:00', '2016-05-31 17:00:00', 1075, denom)
print(err)

0.115087256071
Wall time: 6.68 s


Делаем предсказания для мая.

In [24]:
%%time
Q_may = 0
for region in df.columns:
    res, pred, ids = count_error_region(df.loc[:'2016-05-31 23:00:00'], \
                                        '2016-04-30 23:00:00', '2016-05-31 17:00:00', region, denom)
    Q_may += res
print("\n", Q_may)


20.9557200193
CPU times: user 18min 28s, sys: 3min 13s, total: 21min 42s
Wall time: 12min 29s


Ошибка предсказания на май 2016 года. Она значительно уменьшилась по сравнению с прошлой неделей: было около 30.

In [25]:
print(Q_may)

20.9557200193


Делаем предсказания на июнь по данным до мая 2016 года и считаем ошибку.

In [44]:
%%time
R = 102
H = 715
denom = 1.0/(R*H*6)
Q_june = 0
all_ids = []
all_preds = []
for region in df.columns:
    res, pred, ids = count_error_region(df.loc[:'2016-06-30 23:00:00'], \
                                        '2016-05-31 23:00:00', '2016-06-30 17:00:00', 
                                        region, denom,
                                        '2016-06-01', '2016-06-30')
    Q_june += res
    all_ids.append(ids)
    all_preds.append(pred)
print(Q_june)

20.3926429294
CPU times: user 19min 13s, sys: 2min 24s, total: 21min 37s
Wall time: 11min 54s


In [45]:
pred_df = pd.DataFrame(np.array(all_preds).ravel(), index=np.array(all_ids).ravel(), columns=['y'])
pred_df.index.name = 'id'
pred_df.to_csv("result.csv", sep=',')

In [47]:
pred_df.shape

(437580, 1)